In [10]:
import requests
import pandas as pd

base_url = "https://api-na.hosted.exlibrisgroup.com/almaws/v1"
API_KEY = "l7xx2af7939c63424511946e0fcdc35fe22a"  

def get_users():
    url = f"{base_url}/users?limit=100&apikey={API_KEY}"
    headers = {"Accept": "application/json"}  
    response = requests.get(url, headers=headers)
    data = response.json()  
    users = data.get("user", [])
    return users

def get_user_loans(user_id):
    url = f"{base_url}/users/{user_id}/loans?apikey={API_KEY}"
    headers = {"Accept": "application/json"}  
    response = requests.get(url, headers=headers)
    data = response.json()
    return data.get("item_loan", [])

def get_user_fines(user_id):
    url = f"{base_url}/users/{user_id}/fees?apikey={API_KEY}"
    headers = {"Accept": "application/json"}    
    response = requests.get(url, headers=headers)
    data = response.json()
    total_fine = sum(float(fee.get("balance", 0)) for fee in data.get("fee", []))
    return total_fine


In [11]:
def process_data():
    users = get_users()
    
    for user in users:
        user["loans"] = get_user_loans(user["primary_id"])
        user["total_fines"] = get_user_fines(user["primary_id"])


    users.sort(key=lambda x: x["total_fines"], reverse=True)
    
    return users


In [ ]:
def generate_html(users):
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Library Users with Overdue Loans & Fines</title>
        <style>
            table { width: 100%%; border-collapse: collapse; }
            th, td { border: 1px solid black; padding: 8px; text-align: left; }
            th { background-color: #f2f2f2; }
        </style>
    </head>
    <body>
        <h2>Library Users with Overdue Loans & Fines</h2>
        <table>
            <tr>
                <th>User ID</th>
                <th>Name</th>
                <th>Total Fines ($)</th>
                <th>Loans</th>
            </tr>
    """

    for user in users:
        loan_titles = ", ".join([loan.get("title", "Unknown") for loan in user["loans"]])
        html_content += f"""
            <tr>
                <td>{user["primary_id"]}</td>
                <td>{user["first_name"]} {user["last_name"]}</td>
                <td>{user["total_fines"]:.2f}</td>
                <td>{loan_titles}</td>
            </tr>
        """

    html_content += """
        </table>
    </body>
    </html>
    """

    with open("library_report.html", "w", encoding="utf-8") as f:
        f.write(html_content)


In [16]:
users = process_data()
generate_html(users)


HTML file generated: library_report.html
